# Model Evaluation: Granite as Judge

_Note_: for an introduction to model evaluation, see the [Quick Start](Unitxt_Quick_Start.ipynb) Cookbook.

In this example, we use Granite as an evaluator of predictions created by another model.

## Load Dependencies

In [ ]:
%pip install replicate
%pip install unitxt
%pip install openai
%pip install litellm
%pip install diskcache
%pip install tenacity
%pip install tabulate
%pip install git+https://github.com/ibm-granite-community/utils

and

In [ ]:
from unitxt.api import evaluate, create_dataset
from unitxt.inference import CrossProviderInferenceEngine
from unitxt.llm_as_judge import LLMJudgeDirect, DirectCriteriaCatalogEnum

from ibm_granite_community.notebook_utils import get_env_var


import nest_asyncio
nest_asyncio.apply()

## Set up the sample data and predictions

In [ ]:
data = [
    {"question": "Who is Harry Potter?"},
    {"question": "How can I protect myself from the wind while walking outside?"},
    {"question": "What is a good low cost of living city in the US?"},
]

predictions = [
    """Harry Potter is a young wizard who becomes famous for surviving an attack by the dark wizard Voldemort, and later embarks on a journey to defeat him and uncover the truth about his past.""",
    """You can protect yourself from the wind by wearing windproof clothing, layering up, and using accessories like hats, scarves, and gloves to cover exposed skin.""",
    """A good low-cost-of-living city in the U.S. is San Francisco, California, known for its affordable housing and budget-friendly lifestyle.""",
]

## Define the judge metric

We would like to evaluate how relevant the answer is to the question asked.

In [ ]:
metric = LLMJudgeDirect(
    evaluator_name="GRANITE3_3_8B",
    inference_engine=CrossProviderInferenceEngine(
        model="replicate/ibm-granite/granite-3.3-8b-instruct",
        provider="replicate",
        credentials={'api_token': get_env_var('REPLICATE_API_TOKEN')},
        provider_specific_args={'max_requests_per_second': 1}
    ),
    criteria=DirectCriteriaCatalogEnum.ANSWER_RELEVANCE.value,
    context_fields=["question"],
    criteria_field="criteria",
)

## Create the dataset

In [ ]:
dataset = create_dataset(
    task="tasks.qa.open",
    test_set=data,
    metrics=[metric],
    split="test"
)

## Perform the evaluation

In [ ]:
results = evaluate(predictions=predictions, data=dataset)

## Print the results

In [ ]:

print("Global Scores:")
print(results.global_scores.summary)

print("Instance Scores:")
print(results.instance_scores)